In [1]:
import pandas as pd

In [9]:
qlf = {}

counterYear = 2012
endYear = 2020

while counterYear <= endYear:
  qlf[counterYear] = pd.read_csv(f'raw/{counterYear}.csv')
  counterYear += 1

In [10]:
counterYear = 2012
endYear = 2020

while counterYear <= endYear:
  qlf[counterYear]['Year'] = counterYear
  
  counterYear += 1

In [11]:
frames = []

for key in qlf.keys():
  frames.append(qlf[key])
  
mergedData = pd.concat(frames)

In [12]:
mergedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606 entries, 0 to 79
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Country                         606 non-null    object 
 1   Quality of Life Index           606 non-null    float64
 2   Purchasing Power Index          606 non-null    float64
 3   Safety Index                    606 non-null    float64
 4   Health Care Index               606 non-null    float64
 5   Cost of Living Index            606 non-null    float64
 6   Property Price to Income Ratio  606 non-null    float64
 7   Traffic Commute Time Index      606 non-null    float64
 8   Pollution Index                 606 non-null    float64
 9   Climate Index                   606 non-null    object 
 10  Year                            606 non-null    int64  
dtypes: float64(8), int64(1), object(2)
memory usage: 56.8+ KB


In [13]:
mergedData.loc[mergedData['Climate Index'] == '-', 'Climate Index'] = -1

In [14]:
mergedData['Climate Index'] = mergedData['Climate Index'].astype('float64')

In [62]:
# a meglévő adatok mellett szükségem lesz még az ISO codeokra, s azt hiszem az ország koordinátákra, de ezt még megkell nézzem a korábbi térképes projektjeimnél, a koordináták csak a térkép esetében fontosak

In [15]:
codes = pd.read_json('raw/codes.json', orient='index')
codes = codes.reset_index()
codes = codes.rename(columns={'index': 'Code', 0: 'Country'})

In [16]:
mergedData.loc[mergedData['Country'] == 'Russia', 'Country'] = 'Russian Federation'
mergedData.loc[mergedData['Country'] == 'United States', 'Country'] = 'United States of America'
mergedData.loc[mergedData['Country'] == 'United Kingdom', 'Country'] = 'United Kingdom of Great Britain'
mergedData.loc[mergedData['Country'] == 'Czech Republic', 'Country'] = 'Czechia'
mergedData.loc[mergedData['Country'] == 'Bosnia And Herzegovina', 'Country'] = 'Bosnia and Herzegovina'
mergedData.loc[mergedData['Country'] == 'South Korea', 'Country'] = 'Korea, Republic of'
mergedData.loc[mergedData['Country'] == 'Vietnam', 'Country'] = 'Viet Nam'
mergedData.loc[mergedData['Country'] == 'Iran', 'Country'] = 'Iran (Islamic Republic of)'
mergedData.loc[mergedData['Country'] == 'Moldova', 'Country'] = 'Moldova, Republic of'
mergedData.loc[mergedData['Country'] == 'Venezuela', 'Country'] = 'Venezuela (Bolivarian Republic of)'
mergedData.loc[mergedData['Country'] == 'Bolivia', 'Country'] = 'Bolivia (Plurinational State of)'

In [17]:
mergedData = mergedData.merge(codes, how='left', left_on='Country', right_on='Country')

In [18]:
mergedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606 entries, 0 to 605
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Country                         606 non-null    object 
 1   Quality of Life Index           606 non-null    float64
 2   Purchasing Power Index          606 non-null    float64
 3   Safety Index                    606 non-null    float64
 4   Health Care Index               606 non-null    float64
 5   Cost of Living Index            606 non-null    float64
 6   Property Price to Income Ratio  606 non-null    float64
 7   Traffic Commute Time Index      606 non-null    float64
 8   Pollution Index                 606 non-null    float64
 9   Climate Index                   606 non-null    float64
 10  Year                            606 non-null    int64  
 11  Code                            606 non-null    object 
dtypes: float64(9), int64(1), object(2)
m

In [19]:
mergedData.loc[mergedData['Code'].isnull(), 'Country']

Series([], Name: Country, dtype: object)

In [79]:
mergedData.to_csv('raw/mergedButRaw.csv', index = False)

In [18]:
gdp = pd.read_csv('raw/gdp.csv')
gdp.drop(labels=list(map(lambda x : str(x), range(1960, 2011))), axis=1, inplace=True)
gdp.drop(labels=['Country Name'], axis=1, inplace=True)

In [20]:
gdp = gdp.melt(id_vars=['Country Code'], value_vars=list(map(lambda x : str(x), range(2011, 2021))), var_name='Year', value_name='GDP').sort_values(['Country Code', 'Year'])

In [18]:
gdp = gdp.round({'GDP': 2})
gdp['Year'] = gdp['Year'].astype('int64')

In [19]:
gdp = gdp.merge(mergedData, how='right', left_on=['Country Code', 'Year'], right_on=['Code', 'Year'])

In [20]:
gdp.drop(labels=['Country Code'], axis=1, inplace=True)

In [21]:
normalizedData = gdp

In [151]:
normalizedData.to_csv('normalized/normalized.csv', index=False)
normalizedData.to_json('normalized/normalized.json', orient='records')

In [27]:
hunISO = pd.read_csv('raw/hungarianISO.csv')
hunISO

,Ország / Régió,Numerikus,Alpha-3,Alpha-2,Helyi ISO kódok
0,Afganisztán,4,AFG,AF,ISO 3166-2:AF
1,Åland,248,ALA,AX,ISO 3166-2:AX
2,Albánia,8,ALB,AL,ISO 3166-2:AL
3,Algéria,12,DZA,DZ,ISO 3166-2:DZ
4,Amerikai Csendes-óceáni szigetek,581,UMI,UM,ISO 3166-2:UM
...,...,...,...,...,...
246,Vietnám,704,VNM,VN,ISO 3166-2:VN
247,Wallis és Futuna,876,WLF,WF,ISO 3166-2:WF
248,Zambia,894,ZMB,ZM,ISO 3166-2:ZM
249,Zimbabwe,716,ZWE,ZW,ISO 3166-2:ZW


In [28]:
hunISO.drop(labels=['Helyi ISO kódok', 'Alpha-2', 'Numerikus'], axis=1, inplace=True)

In [29]:
hunISO.rename(columns={'Ország / Régió': 'Country'}, inplace=True)

In [30]:
hunISO.rename(columns={'Alpha-3': 'Code'}, inplace=True)

In [31]:
merged = normalizedData.merge(hunISO, how='left', left_on='Code', right_on='Code')

In [33]:
merged.rename(columns={'Country_x': 'Country_EN', 'Country_y': 'Country_HU'}, inplace=True)

In [35]:
merged.to_csv('normalized/normalized.csv', index=False)
merged.to_json('normalized/normalized.json', orient='records')

In [4]:
import pandas as pd

gni = pd.read_csv('raw/gni.csv')
gni.drop(labels=list(map(lambda x : str(x), range(1960, 2011))), axis=1, inplace=True)

In [8]:
gni.drop(labels=['Indicator Name', 'Indicator Code'], axis=1, inplace=True)

,Country Name,Country Code,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,31000.000000,32030.000000,34610.000000,35420.00000,35740.000000,35880.000000,36750.000000,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,3174.505439,3118.859472,3236.067310,3384.42560,3422.374577,3541.464741,3598.576800,3681.298944,3736.599416,3583.939696
2,Afghanistan,AFG,1700.000000,1920.000000,2020.000000,2070.00000,2110.000000,2000.000000,2090.000000,2100.000000,2190.000000,2100.000000
3,Africa Western and Central,AFW,3417.499600,3532.962685,3762.240464,4051.91937,4021.290868,3959.126893,3972.513854,4048.230818,4177.241642,4096.053208
4,Angola,AGO,6130.000000,6810.000000,7130.000000,7680.00000,6960.000000,6730.000000,6860.000000,6550.000000,6370.000000,5900.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,7970.000000,8300.000000,8580.000000,8830.00000,9510.000000,10130.000000,10650.000000,11200.000000,12070.000000,11660.000000
262,"Yemen, Rep.",YEM,3350.000000,3340.000000,3520.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,South Africa,ZAF,13080.000000,12880.000000,13280.000000,13260.00000,13390.000000,13400.000000,13480.000000,13810.000000,13930.000000,13130.000000
264,Zambia,ZMB,3160.000000,3360.000000,3430.000000,3470.00000,3360.000000,3300.000000,3330.000000,3550.000000,3550.000000,3360.000000


In [20]:
gni = gni.melt(id_vars=['Country Code'], value_vars=list(map(lambda x : str(x), range(2011, 2021))), var_name='Year', value_name='GNI').sort_values(['Country Code', 'Year'])

In [21]:
gni

,Country Code,Year,GNI
0,ABW,2011,31000.0
266,ABW,2012,32030.0
532,ABW,2013,34610.0
798,ABW,2014,35420.0
1064,ABW,2015,35740.0
...,...,...,...
1595,ZWE,2016,2750.0
1861,ZWE,2017,3730.0
2127,ZWE,2018,3950.0
2393,ZWE,2019,3710.0


In [22]:
gni = gni.round({'GDP': 2})
gni['Year'] = gni['Year'].astype('int64')

In [24]:
normalizedData = gni.merge(mergedData, how='right', left_on=['Country Code', 'Year'], right_on=['Code', 'Year'])
normalizedData.drop(labels=['Country Code'], axis=1, inplace=True)

In [26]:
normalizedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606 entries, 0 to 605
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Year                            606 non-null    int64  
 1   GNI                             593 non-null    float64
 2   Country                         606 non-null    object 
 3   Quality of Life Index           606 non-null    float64
 4   Purchasing Power Index          606 non-null    float64
 5   Safety Index                    606 non-null    float64
 6   Health Care Index               606 non-null    float64
 7   Cost of Living Index            606 non-null    float64
 8   Property Price to Income Ratio  606 non-null    float64
 9   Traffic Commute Time Index      606 non-null    float64
 10  Pollution Index                 606 non-null    float64
 11  Climate Index                   606 non-null    float64
 12  Code                            606 

In [34]:
merged

# A GNI/fő utólag lett becserélve a GDP/fő helyett, ezért rendezetlenebb

,Year,GNI,Country_EN,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index,Code,Country_HU
0,2012,61140.0,Switzerland,194.11,127.42,74.22,79.51,148.69,6.53,30.88,35.49,-1.00,CHE,Svájc
1,2012,44400.0,Germany,184.42,114.22,78.80,71.96,92.83,4.80,33.12,45.14,-1.00,DEU,Németország
2,2012,65910.0,Norway,183.43,94.21,80.93,79.04,166.65,7.10,18.50,19.22,-1.00,NOR,Norvégia
3,2012,69160.0,United Arab Emirates,177.07,134.89,64.05,63.89,80.79,4.49,17.00,69.17,-1.00,ARE,Egyesült Arab Emírségek
4,2012,31620.0,New Zealand,174.28,88.59,53.19,86.81,107.37,5.71,25.25,17.08,-1.00,NZL,Új-Zéland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,2020,12860.0,Sri Lanka,85.16,24.45,59.78,72.53,31.61,31.14,59.01,59.14,59.11,LKA,Srí Lanka
602,2020,13300.0,Iran (Islamic Republic of),74.14,22.69,50.75,51.70,39.01,25.11,48.01,77.45,70.99,IRN,Irán
603,2020,4500.0,Kenya,70.56,27.32,38.34,55.59,40.21,31.63,56.65,76.60,99.79,KEN,Kenya
604,2020,5390.0,Bangladesh,70.03,33.21,36.06,42.80,32.25,13.10,56.73,86.21,71.29,BGD,Banglades
